In [1]:
import sys, os
import torch, wandb
import torch.nn as nn
from torch.utils.data import DataLoader
sys.path.append(os.path.abspath(os.path.join(os.curdir, '..')))

In [2]:
from configs import unet_convnextv2_config as config
from models.unet_convnextv2 import Unet
from datasets.depth_dataset import DepthDataset
from utils.train_utils import train_model

/home/v.lomtev/env/python3.12_vlad/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/v.lomtev/env/python3.12_vlad/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
# Set a fixed random seed for reproducibility
torch.manual_seed(config.random_seed)

train_full_dataset = DepthDataset(
    data_dir=os.path.join(config.dataset_path, 'train/train'),
    list_file=os.path.join(config.dataset_path, 'train_list.txt'), 
    transform=config.transform_train,
    target_transform=config.target_transform,
    has_gt=True)
    
    # Create test dataset without ground truth
test_dataset = DepthDataset(
    data_dir=os.path.join(config.dataset_path, 'test/test'),
    list_file=os.path.join(config.dataset_path, 'test_list.txt'),
    transform=config.transform_val,
    has_gt=False)  # Test set has no ground truth
    
# Split training dataset into train and validation
total_size = len(train_full_dataset)
train_size = int((1-config.val_part) * total_size)  
val_size = total_size - train_size    
    
train_dataset, val_dataset = torch.utils.data.random_split(
    train_full_dataset, [train_size, val_size]
)
#val_dataset.transform = config.transform_val # I dont think we need to use augmentations for validation

# Create data loaders with memory optimizations
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.train_bs, 
    shuffle=True, 
    num_workers=config.num_workers, 
    pin_memory=True,
    drop_last=True,
    persistent_workers=True
)
    
    
val_loader = DataLoader(
    val_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)
    
test_loader = DataLoader(
    test_dataset, 
    batch_size=config.val_bs, 
    shuffle=False, 
    num_workers=config.num_workers, 
    pin_memory=True
)

print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")

Train size: 22772, Validation size: 1199, Test size: 650


In [4]:
model = config.model()
# #model = nn.DataParallel(model)


optimizer = config.optimizer(model.parameters())
print(f"Using device: {config.device}")


/home/v.lomtev/env/python3.12_vlad/lib/python3.12/site-packages/segmentation_models_pytorch/decoders/dpt/model.py:118: UserWarning: Encoder does not have prefix tokens (e.g. cls_token), but `decoder_readout` is set to 'cat'. It's recommended to set `decoder_readout='ignore'` when using a encoder without prefix tokens.
  warnings.warn(


Using device: cuda:0


In [5]:
exp_name = "DPT_mixedloss_normalizedlog_2"

In [ ]:
print("Starting training...")
with wandb.init(project="CIL",
                save_code=True,
                notes=config.WANDB_NOTES):
    model = train_model(model, train_loader, val_loader,
                        config.loss, optimizer, config.epochs, config.device,
                       exp_path=os.path.join(config.dataset_path, exp_name), log_input=True)

Starting training...


wandb: Currently logged in as: noloo to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/home/v.lomtev/env/python3.12_vlad/lib/python3.12/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/home/v.lomtev/env/python3.12_vlad/lib/python3.12/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Epoch 1/50


Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:26<00:00,  5.60it/s]


Train Loss: 0.0886, Validation Loss: 0.0490
New best model saved at epoch 1 with validation loss: 0.0490
Epoch 2/50


...

Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:26<00:00,  5.57it/s]


Train Loss: 0.0532, Validation Loss: 0.0373
New best model saved at epoch 3 with validation loss: 0.0373
Epoch 4/50


Evaluating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:26<00:00,  5.64it/s]


Train Loss: 0.0470, Validation Loss: 0.0361
New best model saved at epoch 4 with validation loss: 0.0361
Epoch 5/50


Training:  30%|█████████████████████████████████████████████▋                                                                                                            | 845/2846 [06:21<15:01,  2.22it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Training:  17%|██████████████████████████▎                                                                                                                               | 486/2846 [03:39<17:42,  2.22it/s]

In [ ]:
1

In [ ]:
model.load_state_dict(torch.load(f'{os.path.join(config.dataset_path, exp_name)}/best_model_15.pt'))

In [ ]:
import utils.train_utils as tu

In [ ]:
from utils.train_utils import evaluate_model
import importlib
importlib.reload(tu)
tu.evaluate_model(model, val_loader, config.device,
                  exp_path=os.path.join(config.dataset_path, exp_name), log_input=True)

In [ ]:
from utils.train_utils import evaluate_model
import importlib
importlib.reload(tu)
tu.evaluate_model(model, val_loader, config.device,
                  exp_path=os.path.join(config.dataset_path, exp_name), log_input=True)

In [ ]:
importlib.reload(tu)
tu.generate_test_predictions(model, test_loader, config.device,
                             exp_path=os.path.join(config.dataset_path, exp_name), log_input=True)

In [ ]:
importlib.reload(tu)
tu.visualize_test_predictions(model, test_loader, config.device,
                              exp_path=os.path.join(config.dataset_path, exp_name), log_input=True)

In [ ]:
torch.cuda.empty_cache()